## Outline

1. Modified SN class
2. Overall setup - What is the data, model, task
3. Plotting functions - 3d, contour
4. Individual algorithms and how they perform
5. Exercise

In [1]:
#used for 3D Plotting
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

from matplotlib import cm
import matplotlib.colors

# For animated plots
from matplotlib import animation, rc
from IPython.display import HTML

import numpy as np

In [ ]:
class SN:
  #This is a single sigmoid neuron - takes only a single input
  '''
  We are taking the w_init & b_ init which are the initial value we will like to have for w and b
  algo argument - tells which algo do we want to use for Gradient Descent
  We have a few array/list varibes, suffixed with '_h' this indicated that these are history variables.
  w_h,b_h are self explanatory, e_h tracks the error/loss at the current moment
  algo is being stored in an internal variable.
  '''
  def __init__(self, w_init, b_init, algo):
    self.w = w_init
    self.b = b_init
    self.w_h = []
    self.b_h = []
    self.e_h = []
    self.algo = algo
  
  '''  
  Now the signoid function additionaly takes w, b as inputs. 
  We will be calculating the sigmoid value currently moduled by the class,
  but there is a need to run the sigmoid function for some other values
  so here if w and b if spefied we will use them or else we will use the default
  '''
  def sigmoid(self, x, w=None, b=None):
    if w is None:
      w = self.w
    if b is None:
      b = self.b
    return 1. / (1. + np.exp(-(w*x + b))) # here w and b are scalar inputs
  
  '''  
  Given an X and Y which is a training data that we have and also 
  w,b optionally spefied like the above sigmoid function 
  we would line to compute the MSE- mean squared error    
  '''
  def error(self, X, Y, w=None, b=None):
    if w is None:
      w = self.w
    if b is None:
      b = self.b
    err = 0
    for x, y in zip(X, Y):
      err += 0.5 * (self.sigmoid(x, w, b) - y) ** 2
    return err
  '''
  returns the gradient of the sigoid w.r.t to the input
  We have the ability to use some other values of w and b 
  just as in the case of sigmoid function 
  to calculate the sigmoid at some other fx^n not that we are currently at
  '''
  def grad_w(self, x, y, w=None, b=None):
    if w is None:
      w = self.w
    if b is None:
      b = self.b
    y_pred = self.sigmoid(x, w, b)
    return (y_pred - y) * y_pred * (1 - y_pred) * x

  '''  
  same applies for grad b as in grad a
  The only change is that the * x in return goes away
  therefore it produces the gradienof the loss (in the error fx^n) w.r.t w & b
  '''
  def grad_b(self, x, y, w=None, b=None):
    if w is None:
      w = self.w
    if b is None:
      b = self.b
    y_pred = self.sigmoid(x, w, b)
    return (y_pred - y) * y_pred * (1 - y_pred)
  
  '''  
  The fit fx^n takes X and Y the training data and a bunch of other stuff,
  epochs - no. of times we go over the data, eta - learning rate
  other parameters are comming fromt the different GD learning algo
  thery have been sassigned default value which ofcourse can vbe overwritten
  '''
  def fit(self, X, Y, 
          epochs=100, eta=0.01, gamma=0.9, mini_batch_size=100, eps=1e-8,  
          beta=0.9, beta1=0.9, beta2=0.9
         ):
    self.w_h = []
    self.b_h = []
    self.e_h = []
    self.X = X
    self.Y = Y
    
    # There are many of algos for the different types of Gradient Descent Learning Algos:
    # These algos are avilable in the init fx^n's itself as we saw above
    
    '''
    This is the Gradient decent that we are already familiar with
    we simply iterate over all the data eopchs no. of times
    each case we initialise dw and db value
    for every inpu we accumalate dw and db values, by calling the respective grad fucntions
    then we have the update rule, this take only one parameter for the fit function
    which is the learning rate - eta
    after ebery update to w and b we call the append log functions
    It apppends the current value of weight and bias into the list that we are maintining
    and also calculates the error at the current point
    '''
    if self.algo == 'GD':
      for i in range(epochs):
        dw, db = 0, 0
        for x, y in zip(X, Y):
          dw += self.grad_w(x, y)
          db += self.grad_b(x, y)
        self.w -= eta * dw / X.shape[0]
        self.b -= eta * db / X.shape[0]
        self.append_log()
        
    elif self.algo == 'MiniBatch':
      for i in range(epochs):
        dw, db = 0, 0
        points_seen = 0
        for x, y in zip(X, Y):
          dw += self.grad_w(x, y)
          db += self.grad_b(x, y)
          points_seen += 1
          if points_seen % mini_batch_size == 0:
            self.w -= eta * dw / mini_batch_size
            self.b -= eta * db / mini_batch_size
            self.append_log()
            dw, db = 0, 0
        
    elif self.algo == 'Momentum':
      v_w, v_b = 0, 0
      for i in range(epochs):
        dw, db = 0, 0
        for x, y in zip(X, Y):
          dw += self.grad_w(x, y)
          db += self.grad_b(x, y)
        v_w = gamma * v_w + eta * dw 
        v_b = gamma * v_b + eta * db
        self.w = self.w - v_w
        self.b = self.b - v_b
        self.append_log()
        
    elif self.algo == 'NAG':
      v_w, v_b = 0, 0
      for i in range(epochs):
        dw, db = 0, 0
        v_w = gamma * v_w
        v_b = gamma * v_b
        for x, y in zip(X, Y):
          dw += self.grad_w(x, y, self.w - v_w, self.b - v_b)
          db += self.grad_b(x, y, self.w - v_w, self.b - v_b)
        v_w = v_w + eta * dw
        v_b = v_b + eta * db
        self.w = self.w - v_w
        self.b = self.b - v_b
        self.append_log()
  
  '''It apppends the current value of weight and bias into the list that we are maintining
    and also calculates the error at the current point'''
  def append_log(self):
    self.w_h.append(self.w)
    self.b_h.append(self.b)
    self.e_h.append(self.error(self.X, self.Y))